# Data Generator 

In this notebook we use a small, real dataset to simulate synthetic user data. The data we begin with is the 'last.fm' data set and can be found [here](http://ocelma.net/MusicRecommendationDataset/lastfm-1K.html).  

The data set contains music listening history from 992 users. Specifically, we have access to the songs a user listened to, the recording artist and a timestamp for each listen. 

For the purpose of our application (Minhash for recommendation) it will be enough for us to just have a list of artists listened to by a user, as well as how many times they played that artist.

We begin by loading in the existing data, and extracting the artists played and number of plays for each user. 

In [1]:
import pandas as pd

df = pd.read_parquet("data/music.parquet")

In [2]:
df.sample(10, random_state=1)

,0,1,2
12708245,user_000684,2006-11-20T00:52:10Z,The Tragically Hip
574545,user_000023,2007-11-17T09:00:20Z,Rebekah Del Rio
11024724,user_000590,2008-06-15T20:16:28Z,Explosions In The Sky
7138851,user_000366,2006-11-18T00:59:37Z,Meltiis
7982242,user_000427,2007-12-23T06:33:29Z,Derrick Morgan
7447167,user_000384,2008-07-04T14:30:41Z,Girl Talk
17460811,user_000906,2006-09-18T12:25:52Z,Creedence Clearwater Revisited
15220317,user_000793,2007-09-27T17:38:17Z,Lightning Bolt
13575207,user_000714,2009-03-16T06:26:37Z,Britney Spears
2938512,user_000147,2006-05-08T19:43:47Z,Broadcast


In [3]:
users = df['0'].unique()
dusers = {x+1:y for x,y in enumerate(sorted(set(users)))}
len(dusers)

992

For each user, we want to compute the unique artists they listened to.
and generate minhash signatures for them

In [4]:
from datasketching.minhash import SimpleMinhash
from datasketching.minhash import murmurmaker

In [5]:
def generate_minhash_sig(user_dat, nhash):
    mh = SimpleMinhash(nhash)
    for row in user_dat:
        mh.add(row)
    return mh

def unique_artists(df):
    return df['2'].unique()

In [6]:
grouped_df = df.groupby(['0'])

In [7]:
un_artists = grouped_df.apply(unique_artists)
mh_sigs = un_artists.apply(generate_minhash_sig, nhash = 128)

In [8]:
len(un_artists[0])

657

from each user we are going to generate 100 pseudo users. For user x we do this by: 
    
    1. sampling 70% of user x's listening history 
    2. finding the 10 most similar users from the existing set of users
        a. sample 6 of these
        b. sample 5% of each of these user's listening history (excluding *everything* that user x listened to, and everything already sampled from other users.)
    3. apply a poisson rv to their history.


In [9]:
x = mh_sigs[0]
artists_listened = len(un_artists[0])
sim=[]
for mh in range(0, 992):
    sim.append(mh_sigs[mh].similarity(mh_sigs[0]))

similar = set(sorted(sim, reverse = True)[1:11])
similar
similar_users = ([i for i, e in enumerate(sim) if e in similar])

In [10]:
similar_users

[52, 77, 355, 383, 446, 501, 836, 840, 867, 937]

In [11]:
from random import sample 

selected = sample(similar_users, 6)

In [12]:
selected

[77, 383, 840, 446, 836, 355]

In [13]:
#### take a subsample of 70% of user 0's listening history. 

In [14]:
import numpy as np

####### all of that user's listening history 

heard = un_artists[0]
listened = []
to_sample = int(np.floor(artists_listened)*0.05)
for u in selected:
    print(len(listened))
    possible = np.setdiff1d(un_artists[u], (list(un_artists[0])+listened))
    listened = listened + list(np.random.choice(un_artists[u], size=to_sample, replace = False))
    print(len(possible))

listened = listened + list(np.random.choice(un_artists[0], size=int(np.floor(artists_listened*0.7))))



0
1257
32
1343
64
2895
96
3820
128
3216
160
742


In [15]:
len(listened)

651

In [16]:
def user_data(user, grouped_data, dusers):
    return grouped_data.get_group(dusers[user]) 

In [17]:

def top_k_listens(listening_history, k=artists_listened):
    hist = listening_history.groupby(['2'])
    return hist.count().sort_values(by='0', ascending=False) #.head(k).index.values

In [18]:
u_0 = user_data(1, grouped_df, dusers)

In [19]:
listen_vals = top_k_listens(u_0)['1'].values

In [20]:
## that's the number of listens. 

In [21]:
user_plays = np.random.choice(listen_vals, size=len(listened), replace = False)

In [22]:
user_data = {'user':np.repeat('user_0',len(listened), axis=0) , 'artist':listened, 'plays':user_plays} 
user_df = pd.DataFrame(user_data) 

In [23]:
user_df

,user,artist,plays
0,user_0,Cellar 55,1
1,user_0,Daft Punk Vs. Jamiroquai,1
2,user_0,Koop,38
3,user_0,Midfield General,5
4,user_0,Mambana,19
5,user_0,The Cure,3
6,user_0,Loscil,1
7,user_0,Gentleman & Morgan Heritage,1
8,user_0,Phats & Small,11
9,user_0,Usher,13


# Now wrap this in a function to do this multiple times for each user. 

In [24]:
#for u in range(0,len(dusers)):
for u in range(0, 5):
    print(u)
    x = mh_sigs[u]
    artists_listened = len(un_artists[u])
    to_sample = int(np.floor(artists_listened)*0.05)
    sim=[]
    for mh in range(0, 992):
        sim.append(mh_sigs[mh].similarity(mh_sigs[0]))
    
    similar = set(sorted(sim, reverse=True)[1:11]) # the ten largest similarities
    similar_users = ([i for i, e in enumerate(sim) if e in similar]) # extract the user values
    
    
    user_play_fr = grouped_df.get_group(dusers[(u+1)]).groupby(['2']).count()['1'].values
    
    
    for j in range(0, 5):
        print(j)
        ### make 5 new users for each user
        username = 'user'+str(u)+str(j)
        print(username)
        selected = sample(similar_users, 6)
        listened = []
        for k in selected:
            possible = np.setdiff1d(un_artists[k], (list(un_artists[u])+listened))
            listened = listened + list(np.random.choice(un_artists[k], size = to_sample, replace = False))
            
        listened = listened + list(np.random.choice(un_artists[u], size=int(np.floor(artists_listened*0.7))))
        
        ### now simulate user plays. 
        user_plays = np.random.choice(user_play_fr, size=len(listened), replace = False)
        
        user_data = {'user':np.repeat(username,len(listened), axis=0) , 'artist':listened, 'plays':user_plays} 
        user_df = pd.DataFrame(user_data) 


0
0
user00
1
user01
2
user02
3
user03
4
user04
1
0
user10
1
user11
2
user12
3
user13
4
user14
2
0
user20
1
user21
2
user22
3
user23
4
user24
3
0
user30
1
user31
2
user32
3
user33
4
user34
4
0
user40
1
user41
2
user42
3
user43
4
user44


to do - concatonate output
write output to file
load back in and run in minhash notebook. 